In [43]:
from inspect import getsource
import errno
import os
import signal
import functools

import yfinance as yf
import numpy as np
import pandas as pd

In [60]:
def get_weights(n):
    """
        Returns a vector of size n, with weights, the sum should be 1.
    """

    search_space = np.linspace(0, 1, 1_000_000)
    cumulative_weights = 0
    vector_weight = []

    for i in range(n - 1):
        weight = np.random.choice(list(search_space)) ### uniform distribution.
        vector_weight.append(weight)
        cumulative_weights = cumulative_weights + weight
        search_space = np.linspace(0, 1 - cumulative_weights, 1_000_000)

    last_weight = 1 - cumulative_weights
    vector_weight.append(last_weight)
    return vector_weight

def createweightmatrix(n):
    """
        Returns a Dataframe with n rows and 10 columns, with weights.
    """
    weights = []

    for i in range(n):
        individual_weights = get_weights(n)
        weights.append(individual_weights)
    
    portfolio_weights = pd.DataFrame(
        weights, 
        columns=[f'weight_{i}' for i in range(1, n+1)]
    )

    return portfolio_weights

In [61]:
class TimeoutError(Exception):
    pass

In [62]:
def timeout(seconds=15, error_message=os.strerror(errno.ETIME)):
    """
        Decorator for limiting the time execution for a given function.
    """
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

@timeout(5)
def list_wikipedia_sp500() -> pd.DataFrame:
    """
        Get components of the S&P 500 index
    """
    url = 'https://en.m.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp_list = pd.read_html(url, attrs={'id': 'constituents'}, index_col='Symbol')[0].index.to_list()
    sp_set = set(sp_list) - {"BRK.B", "BF.B"}
    sp_list = list(sp_set)    
    return sp_list

In [63]:
START_DATE = '2021-01-01'
sample_size = 8
sp_stocks = list_wikipedia_sp500()
sp_stocks_sample = list(np.random.choice(sp_stocks, sample_size))

In [64]:
sp_market_data = yf.download(
    sp_stocks_sample, 
    start=START_DATE
).drop(
    ["Open","Low","Close","High", "Volume"], 
    axis=1
)

[*********************100%%**********************]  8 of 8 completed


In [65]:
returnslog = np.log(sp_market_data)
compoundedreturns = returnslog.diff()
cretunrsmean = compoundedreturns.dropna().mean(axis=0)
cretunrsmeandf = cretunrsmean.to_frame()
expectedreturn = (np.exp(cretunrsmeandf))-1

In [66]:
mweights = createweightmatrix(10)

In [67]:
mweights

,weight_1,weight_2,weight_3,weight_4,weight_5,weight_6,weight_7,weight_8,weight_9,weight_10
0,0.957380,0.014649,0.023964,0.001968,0.000051,4.939875e-04,7.256846e-04,9.876102e-05,4.440801e-04,2.258102e-04
1,0.695831,0.179686,0.029722,0.073217,0.008880,9.417586e-04,8.989577e-03,7.001340e-04,1.588388e-03,4.441325e-04
2,0.690800,0.189166,0.079914,0.029283,0.009970,3.525065e-04,8.856808e-05,4.848113e-05,2.943168e-06,3.742890e-04
3,0.357673,0.168851,0.188257,0.219485,0.053329,3.983853e-03,2.126293e-03,5.326200e-03,6.509966e-04,3.177978e-04
4,0.653580,0.346117,0.000283,0.000010,0.000009,5.156171e-07,1.809423e-07,8.129645e-08,1.038691e-06,1.781165e-07
5,0.844943,0.121406,0.026724,0.001645,0.005203,7.308645e-05,4.674774e-06,7.418372e-08,9.129062e-07,5.524690e-07
6,0.239440,0.306952,0.085843,0.234306,0.034415,6.005151e-02,2.309557e-02,6.594833e-03,3.212758e-03,6.089302e-03
7,0.116533,0.279842,0.217116,0.278299,0.020117,7.735147e-03,2.876973e-02,4.580625e-03,4.567783e-02,1.329171e-03
8,0.734067,0.224501,0.009348,0.026294,0.005717,4.837959e-05,2.129565e-05,1.757821e-06,1.321366e-06,4.848799e-07
9,0.128964,0.118022,0.263342,0.168880,0.216126,8.818643e-02,4.507978e-03,9.482932e-03,1.938704e-03,5.502097e-04
